In [ ]:
from langchain.document_loaders import DirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
import chromadb
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

In [ ]:
loader = DirectoryLoader('Data', glob="**/*.txt")
docs = loader.load()

In [ ]:
embeddings = OllamaEmbeddings(model="llama3.1")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,)

In [ ]:
document = text_splitter.split_documents(docs)

In [ ]:
document

In [ ]:
global_embeddings = [ embeddings.embed_query(txt) for txt in document]

In [ ]:
# running chromadb locally
client = chromadb.HttpClient(host= "localhost", port= 8000)

In [ ]:
collection = client.create_collection(name="rag_dataset_cosine",metadata={"hnsw:space": "cosine"})

In [ ]:
client.list_collections()

In [ ]:
client.get_collection('rag_dataset')

In [ ]:
# for existing database
collection = client.get_collection('rag_dataset')

In [ ]:
def creating_unique_chunk_ids():
    
    last_index = 0
    last_page_id = None
    for doc in document:
        source = doc.metadata.get('source')
        page = doc.metadata.get('page')
        current_page_id = f"{source}:{page}"
        if current_page_id == last_page_id:
            index += 1

        else:
            index = 0
        
        last_page_id = current_page_id
        chunk_id = f"{source}:{index}"
        doc.metadata["id"] = chunk_id
    return chunk_id

In [ ]:
creating_unique_chunk_ids()

In [ ]:
new_chunk_id = [chunk.metadata['id'] for chunk in document]

In [ ]:
page_content = [page.page_content for page in document]
page_content

In [ ]:
collection.add(documents= page_content, ids= new_chunk_id,embeddings = global_embeddings)

In [ ]:
collection.count()

In [ ]:
collection.peek()

In [ ]:
embedded_querry = embeddings.embed_query("who crowned with plump shrimp and tender calamari")

In [ ]:
querry_text = "who crowned with plump shrimp and tender calamari"

In [ ]:
res = collection.query(embedded_querry,n_results=5,
            include=['distances','embeddings', 'documents', 'metadatas'])

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [ ]:
context_text = "\n\n---\n\n".join([doc for doc in res['documents'][0]])

In [ ]:
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=querry_text)

In [ ]:
print(prompt)

In [ ]:
model = Ollama(model="llama3.1")
response_text = model.invoke(prompt)

In [ ]:
response_text